In [4]:
from transformers import BertTokenizer,BertModel
from torch.utils.data import Dataset,DataLoader
from datasets import load_from_disk
import torch

token = BertTokenizer.from_pretrained(r"D:\github file\Pytorch-LiMu\hugging face\model\google-bert\bert-base-chinese\models--google-bert--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
premodel = BertModel.from_pretrained(r"D:\github file\Pytorch-LiMu\hugging face\model\google-bert\bert-base-chinese\models--google-bert--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f").to(device)
epoch = 2

class Mydataset(Dataset):
    def __init__(self,split):
        self.dataset = load_from_disk(r"D:/github file/Pytorch-LiMu/hugging face/dataset")
        self.dataset = self.dataset[split]
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,item):
        text = self.dataset[item]['text']
        label = self.dataset[item]['label']
        return text,label

class myModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768,2)
    
    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = premodel(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:,0])
        out = out.softmax(dim = 1) #(1,2)/(2,)
        return out


# 自定义函数对数据进行编码
def embedding(data):
    sentes = [i[0] for i in data]
    labels = [i[1] for i in data]

    data = token.batch_encode_plus(
        batch_text_or_text_pairs = sentes,
        add_special_tokens=True, #是否添加特殊token（如[CLS]和[SEP]），默认为True。
        max_length=350,#编码的最大长度（截断或填充）。
        truncation=True,
        padding='max_length', 
        # truncation 是否截断到max_length（可选True/False/'longest_first'等）。
        return_tensors="pt", 
        # return_attention_mask：是否返回attention mask（默认为True）。
        # return_token_type_ids：是否返回token type IDs（默认为True）。
    )

    input_ids = data["input_ids"]
    attention_mask = data["attention_mask"]
    token_type_ids = data["token_type_ids"]
    label = torch.LongTensor(labels)
    return input_ids,attention_mask,token_type_ids,label

train_dataset = Mydataset("train")
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 32,
    shuffle = True,
    drop_last = True,
    collate_fn = embedding
)

model = myModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
loss_fc = torch.nn.CrossEntropyLoss()

model.train()
for e in range(epoch):
    for i,(input_ids,attention_mask,token_type_ids,labels) in enumerate(train_loader):#将数据放到DEVICE上
        input_ids, attention_mask, token_type_ids, labels = input_ids.to(device),\
            attention_mask.to(device),token_type_ids.to(device),labels.to(device)
    #执行前向计算得到输出
        out = model(input_ids,attention_mask, token_type_ids)
        loss = loss_fc(out,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%5 ==0:
            out = out.argmax(dim=1)
            acc =(out == labels).sum().item()/len(labels)
            print(i,loss.item(),acc)#保存模型参数
    torch.save(model.state_dict(),f"{epoch}bert.pt")
    print(e)


0 0.7134514451026917 0.46875
5 0.6531563997268677 0.65625
10 0.5934051275253296 0.90625
15 0.6269028782844543 0.625
20 0.6255695819854736 0.6875
25 0.5593469142913818 0.8125
30 0.5501055121421814 0.8125
35 0.5296679735183716 0.875
40 0.549964964389801 0.8125
45 0.4904342591762543 0.875
50 0.5259115099906921 0.875
55 0.4688456356525421 0.90625
60 0.5442495346069336 0.8125
65 0.5228625535964966 0.84375
70 0.48210766911506653 0.9375
75 0.47963353991508484 0.90625
80 0.492077112197876 0.84375
85 0.4768310487270355 0.90625
90 0.45832765102386475 0.875
95 0.49877506494522095 0.8125
100 0.47444072365760803 0.90625
105 0.4646175801753998 0.90625
110 0.4031294584274292 0.9375
115 0.5296034216880798 0.8125
120 0.45010003447532654 0.96875
125 0.4417475759983063 0.90625
130 0.4535215497016907 0.875
135 0.4887517988681793 0.8125
140 0.4233162999153137 0.9375
145 0.5114350914955139 0.84375
150 0.4007081389427185 0.96875
155 0.4612414538860321 0.875
160 0.49602797627449036 0.84375
165 0.4594311416149

In [5]:
from transformers import BertTokenizer,BertModel
from torch.utils.data import Dataset,DataLoader
from datasets import load_from_disk
import torch

token = BertTokenizer.from_pretrained(r"D:\github file\Pytorch-LiMu\hugging face\model\google-bert\bert-base-chinese\models--google-bert--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
premodel = BertModel.from_pretrained(r"D:\github file\Pytorch-LiMu\hugging face\model\google-bert\bert-base-chinese\models--google-bert--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f").to(device)

class Mydataset(Dataset):
    def __init__(self,split):
        self.dataset = load_from_disk(r"D:/github file/Pytorch-LiMu/hugging face/dataset")
        self.dataset = self.dataset[split]
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,item):
        text = self.dataset[item]['text']
        label = self.dataset[item]['label']
        return text,label

class myModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768,2)
    
    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = premodel(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:,0])
        out = out.softmax(dim = 1) #(1,2)/(2,)
        return out


# 自定义函数对数据进行编码
def embedding(data):
    sentes = [i[0] for i in data]
    labels = [i[1] for i in data]

    data = token.batch_encode_plus(
        batch_text_or_text_pairs = sentes,
        add_special_tokens=True, #是否添加特殊token（如[CLS]和[SEP]），默认为True。
        max_length=350,#编码的最大长度（截断或填充）。
        truncation=True,
        padding='max_length', 
        # truncation 是否截断到max_length（可选True/False/'longest_first'等）。
        return_tensors="pt", 
        # return_attention_mask：是否返回attention mask（默认为True）。
        # return_token_type_ids：是否返回token type IDs（默认为True）。
    )

    input_ids = data["input_ids"]
    attention_mask = data["attention_mask"]
    token_type_ids = data["token_type_ids"]
    label = torch.LongTensor(labels)
    return input_ids,attention_mask,token_type_ids,label

train_dataset = Mydataset("test")
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 32,
    shuffle = True,
    drop_last = True,
    collate_fn = embedding
)

model = myModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
loss_fc = torch.nn.CrossEntropyLoss()



acc = 0
total = 0
model.load_state_dict(torch.load("bert.pt"))
model.eval()
for i,(input_ids,attention_mask,token_type_ids,labels) in enumerate(train_loader):#将数据放到DEVICE上
    input_ids, attention_mask, token_type_ids, labels = input_ids.to(device),\
        attention_mask.to(device),token_type_ids.to(device),labels.to(device)
    #执行前向计算得到输出
    out = model(input_ids,attention_mask, token_type_ids)


    out = out.argmax(dim=1)
    acc += (out == labels).sum().item()
    total += len(labels)
print(acc/total)


0.8868243243243243
